# __Assisted Practice: Training Deep Neural Networks on TensorFlow__
Building Deep Neural Networks on TensorFlow refers to the process of designing and constructing neural network models using the TensorFlow framework. This involves defining the architecture of the neural network, selecting appropriate layers and activation functions, specifying the optimization algorithm, and training the model using data.

Let's understand how to build and train a neural network using TensorFlow.



## Steps to be followed:
1. Import the required libraries
2. Load and inspect the data
3. Build the model
4. Train the model

### Step 1: Import the required libraries

- Import Pandas and NumPy packages.
- Import the TensorFlow package, which is used for text-based applications, image recognition, voice search, and many more.
- Import the Python package cv2, which is used for computer vision and image processing.
- Import the Python package matplotlib, which sets the padding between and around the subplots as well as the figure size.
- Import necessary libraries and modules for building a deep learning model using TensorFlow. It includes modules for convolutional and pooling layers, dropout, flattening, and dense layers.
- Import other libraries for data manipulation, visualization, and image processing.

In [1]:
# Import TensorFlow and required layers for building the model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Import other necessary libraries for data manipulation, visualization, and image processing
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import IPython
from six.moves import urllib

### Step 2: Load and inspect the data


- Load the California Housing dataset from **`fetch_california_housing`**.
- Split the dataset into two sets: the training set **train_features** and **train_labels** and the testing set **test_features** and **test_labels**.
- The testing set is used to evaluate the trained model's performance.


In [2]:
# Load the California Housing dataset from sklearn
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

# Convert to pandas DataFrame
data = pd.DataFrame(housing.data, columns=housing.feature_names)
data['target'] = housing.target

# Split the dataset into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data[housing.feature_names], data['target'], test_size=0.2, random_state=42)

In [3]:
# Standardize the features
scaler = StandardScaler()
train_features = scaler.fit_transform(train_data)
test_features = scaler.transform(test_data)

# Print the standardized training features
print(train_features)

[[-0.326196    0.34849025 -0.17491646 ...  0.05137609 -1.3728112
   1.27258656]
 [-0.03584338  1.61811813 -0.40283542 ... -0.11736222 -0.87669601
   0.70916212]
 [ 0.14470145 -1.95271028  0.08821601 ... -0.03227969 -0.46014647
  -0.44760309]
 ...
 [-0.49697313  0.58654547 -0.60675918 ...  0.02030568 -0.75500738
   0.59946887]
 [ 0.96545045 -1.07984112  0.40217517 ...  0.00707608  0.90651045
  -1.18553953]
 [-0.68544764  1.85617335 -0.85144571 ... -0.08535429  0.99543676
  -1.41489815]]


 __Observation:__


- Here, we can see a few datasets from train dataset.
- The given array represents a multi-dimensional array containing numerical values.
- Each row in the array corresponds to a set of features or data points, while each column represents a specific feature or variable.

### Step 3: Build the Model
Building the neural network requires:
- Configuring the layers of the model and compiling the model.
- Stacking a few layers together using **keras.Sequential**.
- Configuring the loss function, optimizer, and metrics to monitor.
These are added during the model's compile step.



Terminologies:
- The **Loss** function measures how accurate the model is during training; we want to minimize this with the optimizer.
- One must **Optimize** how the model is updated based on the data it sees and its loss function.
- **Metrics** are used to monitor the training and testing steps.

In [4]:
# Function to build the neural network model
def build_model():
    model = keras.Sequential([
        # Input layer with 20 units and ReLU activation
        Dense(20, activation=tf.nn.relu, input_shape=[train_features.shape[1]]),
        # Output layer with 1 unit
        Dense(1)
    ])

    # Compile the model with Adam optimizer and mean absolute error loss
    model.compile(optimizer=tf.optimizers.Adam(),
                  loss='mae',
                  metrics=['mean_absolute_error'])
    return model


### Step 4: Train the model
Training the neural network model requires the following steps:


- Define a custom callback class **PrintDot**, which prints a dot for every epoch during training.

  `PrintDot` is a custom callback class in Keras that is used to provide visual feedback during the training process of a neural network. It prints a dot (.) for every epoch completed, and it prints a new line every 100 epochs. This helps in monitoring the progress of the training process in a simple and visual way without overwhelming the console with too much information.

- Create an instance of the model using the **build_model** function.

- Create an instance of EarlyStopping callback, which monitors the validation loss and stops training if it doesn't improve after a certain number of epochs (specified by patience).

- Train the model using the training features and labels. It runs for 200 epochs, with a validation split of 0.1 (10% of the training data used for validation). The callbacks parameter includes **early_stop** and **PrintDot** callbacks.

- Create a Pandas **DataFrame hist** from the history object returned by the model.fit method. It contains the recorded training and validation metrics.

- Extract the last value of the validation mean absolute error (MAE) from the hist DataFrame and assign it to the variable mae_final.

- Print the final MAE on the validation set, rounded to three decimal places.

In [6]:
# Custom callback class to print a dot for every epoch
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

# Build the model using the build_model function
model = build_model()

# Early stopping callback to stop training if validation loss doesn't improve for 50 epochs
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

# Train the model with training data, using 10% of the data for validation
history = model.fit(train_features, train_labels, epochs=200, verbose=0, validation_split=0.1,
                    callbacks=[early_stop, PrintDot()])

# Create a Pandas DataFrame from the training history
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

# Extract the final mean absolute error from the validation set
mae_final = float(hist['val_mean_absolute_error'].iloc[-1])

print()
print('Final Mean Absolute Error on validation set: {}'.format(round(mae_final, 3)))




....................................................................................................
....................................................................................................
Final Mean Absolute Error on validation set: 0.585


**Observation:**

As shown, the final mean absolute error on the validation set is 0.369.


- Evaluate the model's performance on the normalized test features and prints the mean absolute error (MAE) on the test set.

In [12]:
# Evaluate the model's performance on the test set
mae, _ = model.evaluate(test_features, test_labels)
print('Mean Absolute Error on test set: {}'.format(round(mae, 3)))

129/129 [==============================] - 0s 1ms/step - loss: 0.3639 - mean_absolute_error: 0.3639
Mean Absolute Error on test set: 0.364


**Observation:**

The output indicates the following:

- The evaluation was performed on the `test_features` and `test_labels`.
- The mean absolute error on the test set is also 0.3576.
- The mean absolute error, when rounded, is 0.358.

